In [434]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
from datetime import datetime
from math import floor
import json

In [435]:
starting_time = datetime.now()
time_limit = 5*60

Read data

In [436]:
import re

def parse_dzn(file_path):
    data = {}
    with open(file_path, 'r') as file:
        content = file.read()
        
        # Find all key-value pairs
        key_value_pairs = re.findall(r'(\w+)\s*=\s*(.*?);', content, re.DOTALL)
        
        for key, value in key_value_pairs:
            # Remove any extra whitespace and newlines
            value = value.strip().replace('\n', '')

            #Parse matrix
            if key == 'D':
                rows = value.split('|')
                D = []
                for i in range(1,len(rows)-1):
                    D.append([int(v.strip()) for v in rows[i][1:-1].split(',')])
                value = D
            # Parse arrays
            elif value.startswith('[') and value.endswith(']'):
                value = value[1:-1].split(',')
                value = [int(v.strip()) for v in value]
            else:
                value = int(value)
                
            data[key] = value
    return data

# Usage
'''
file_path = 'inst08.dzn'  # Path to your .dzn file
parsed_data = parse_dzn(file_path)
print(parsed_data)
'''

"\nfile_path = 'inst08.dzn'  # Path to your .dzn file\nparsed_data = parse_dzn(file_path)\nprint(parsed_data)\n"

In [437]:
def read_dat_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Read m and n
    m = int(lines[0].strip())
    n = int(lines[1].strip())
    
    # Read l vector
    l = list(map(int, lines[2].strip().split()))
    
    # Read s vector
    s = list(map(int, lines[3].strip().split()))
    
    # Read D matrix
    D = []
    for line in lines[4:]:
        D.append(list(map(int, line.strip().split())))
    
    return {
        'm': m,
        'n': n,
        'l': l,
        's': s,
        'D': D
    }

Create the model

In [438]:
model = gp.Model()

Parse the instance

In [439]:
innum = input('Type instance number:\n>')
file_path = str(f'inst{innum}.dat')
parsed_data = read_dat_file('Instances/'+file_path)

m = parsed_data['m']
n = parsed_data['n']
s = parsed_data['s']
l = sorted(parsed_data['l'], reverse=True)
D = parsed_data['D']

maxD = np.max(D)
minD = np.min(D)
heuristic_number_of_nodes_per_courier = n//m +3

In [440]:
#s = sorted(s,reverse=True)

Define the variables

In [441]:
max_total_dist = model.addVar(vtype=gp.GRB.INTEGER, name='MaxTotalDist')

cumulative_dist = model.addVars(m, n+1, vtype=gp.GRB.INTEGER, name='CumulativeDist')
cumulative_load = model.addVars(m, n, lb=0, ub=n*max(s), vtype=gp.GRB.INTEGER, name='CumulativeLoad')

load = model.addVars(m, lb=0, vtype=gp.GRB.INTEGER, name='Load')
dist = model.addVars(m, lb=0, vtype=gp.GRB.INTEGER, name='Dist')

paths = model.addVars(m, n+1, n+1, vtype=gp.GRB.BINARY, name='Paths')

u = model.addVars(m, n, vtype=gp.GRB.INTEGER, name='u')

In [442]:
#Objective Function Boundaries
initial_upper_bound = heuristic_number_of_nodes_per_courier * maxD
initial_lower_bound = heuristic_number_of_nodes_per_courier * minD

model.addConstr( max_total_dist <= initial_upper_bound)
model.addConstr( max_total_dist >= initial_lower_bound)

#Callback function for dynamic boundaries
def update_upper_bound(model, where):
    if where == GRB.Callback.MIPSOL:
        # Get the current value of the objective function
        current_obj_val = model.cbGet(GRB.Callback.MIPSOL_OBJ)
        
        # Replace the upper bound with the new objective value
        model.cbLazy(max_total_dist <= current_obj_val)

model.Params.lazyConstraints = 1

Set parameter LazyConstraints to value 1


Define Objective function

In [443]:
model.setObjective(max_total_dist, gp.GRB.MINIMIZE)

Define Constraints

In [444]:
#Maximum total distance is equal to the maximum total distance computed in the final depot "slot" for each courier
#model.addConstrs( gp.quicksum(cumulative_dist[c,j] for j in range(n+1)) <= max_total_dist for c in range(m))
model.addConstrs( dist[c] <= max_total_dist for c in range(m))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

In [445]:
#Every node must be visited exactly once (except for the depot n+1)
for dest in range(n):
    model.addConstr(gp.quicksum(paths[c,source,dest] for c in range(m) for source in range(n+1)) <= 1)
    model.addConstr(gp.quicksum(paths[c,source,dest] for c in range(m) for source in range(n+1)) >= 1)
    #model.addConstr(gp.quicksum(paths[c,source,dest] for c in range(m) for source in range(n+1)) == 1)

In [446]:
for c in range(m):
    #Every courier must visit the depot as last node
    model.addConstr(gp.quicksum(paths[c,source,n] for source in range(n)) == 1)

    #Cumulative distance update
    model.addConstr(dist[c] >= (gp.quicksum(D[source][dest]*paths[c,source,dest] for source in range(n+1) for dest in range(n+1)) ))

    #Cumulative load update
    model.addConstr(load[c] >= (gp.quicksum(s[dest]*paths[c,source,dest] for dest in range(n) for source in range(n+1))) ) #npe

    #Each courier must start from depot
    #model.addConstr(gp.quicksum(paths[c,n,dest] for dest in range(n)) == 1)
    model.addConstr(gp.quicksum(paths[c,n,dest] for dest in range(n)) >= 1)
    model.addConstr(gp.quicksum(paths[c,n,dest] for dest in range(n)) <= 1)

    for source in range(n+1):
        #Couriers cannot stay still
        model.addConstr(paths[c,source,source] <= 0.5) #npe

        for dest in range(n+1):
            #Cumulative distance update
            #model.addConstr((paths[c,source,dest] == 1) >> (cumulative_dist[c,dest] == D[source][dest])) #npe
            #model.addConstr((cumulative_dist[c,dest] == D[source][dest]*paths[c,source,dest])) #npe

            #Path contiguity
            model.addConstr(gp.quicksum(paths[c,source,j] for j in range(n+1)) == (gp.quicksum(paths[c,j,source] for j in range(n+1))))

            #model.addConstr((paths[c,source,dest] == 1) >> (gp.quicksum(paths[c,j,source] for j in range(n+1)) >= 1))
            #model.addConstr((paths[c,source,dest] == 1) >> (gp.quicksum(paths[c,dest,j] for j in range(n+1)) >= 1))

            #Load update
            #if dest < n:
                #model.addConstr((paths[c,source,dest] == 1) >> (cumulative_load[c,dest] == s[dest])) #npe

        #Just in order to speed up the search    
        model.addConstr(gp.quicksum(paths[c,source,j] for j in range(n+1)) <= 1)
        model.addConstr(gp.quicksum(paths[c,j,source] for j in range(n+1)) <= 1)

In [447]:
#Loop Avoidance 3
for c in range(m):  # for each courier
    for source in range(n):
        for dest in range(n):
            if source != dest:
                model.addConstr(u[c, source] - u[c, dest] + n * paths[c, source, dest] <= n - 1)

In [448]:
#Symmetry breaking constraints
for c in range(m-1):
    model.addConstr( gp.quicksum(paths[c,source,dest] for source in range(n) for dest in range(n)) >= \
                     gp.quicksum(paths[c+1,source,dest] for source in range(n) for dest in range(n)) )

'''
for c1 in range(m):
    for c2 in range(c1+1,m):
        model.addConstr(dist[c1]-dist[c2] <= 2*maxD )

for c in range(m-1):
    model.addConstr(dist[c] >= dist[c+1])
'''

'\nfor c1 in range(m):\n    for c2 in range(c1+1,m):\n        model.addConstr(dist[c1]-dist[c2] <= 2*maxD )\n\nfor c in range(m-1):\n    model.addConstr(dist[c] >= dist[c+1])\n'

In [449]:
#Maximum load
for c in range(m):
    #model.addConstr(gp.quicksum(cumulative_load[c,j] for j in range(n)) <= l[c])
    model.addConstr(load[c] <= l[c])

In [450]:
preprocessing_time = datetime.now() - starting_time
safe_bound = 5
model.setParam('TimeLimit', time_limit - preprocessing_time.seconds - safe_bound)

Set parameter TimeLimit to value 293


In [451]:
model.optimize(update_upper_bound)

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 13949 rows, 7345 columns and 719496 nonzeros
Model fingerprint: 0xa0471c2b
Variable types: 0 continuous, 7345 integer (6912 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+03]
  RHS range        [5e-01, 4e+03]
Presolve removed 7117 rows and 144 columns
Presolve time: 0.12s
Presolved: 6832 rows, 7201 columns, 68177 nonzeros
Variable types: 0 continuous, 7201 integer (6768 binary)

Root relaxation: objective 3.020000e+02, 2084 iterations, 0.07 seconds (0.14 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  302.00000    0   90          -  302.00000      -     -    0s
     0     0  

In [452]:
for c in range(m):
    print(c)
    rows = [i for i in range(n+1)]
    adj_matrix = pd.DataFrame(columns=rows, index=rows)
    for i in rows:
        for j in rows:
            adj_matrix.loc[i,j] = int(paths[c,i,j].X)
    print(adj_matrix)
    print('\n')

0
   0  1  2  3  4  5  6  7  8  9   ... 38 39 40 41 42 43 44 45 46 47
0   0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0  0
1   0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0  0
2   0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0  0
3   0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0  0
4   0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0  0
5   0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0  0
6   0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0  0
7   0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0  0
8   0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0  0
9   0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0  0
10  0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0  0
11  0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0  0
12  0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0  0
13  0  0  0  0  0  0  0  0  0  0  ...  0  0  0

In [453]:
def find_path(adj_matrix, courier):
    res = []
    source = n
    while True:
        for dest in range(n+1):
            if adj_matrix[courier, source, dest].X == 1:
                if dest == n:
                    return res
                res.append(dest)
                source = dest
                break

In [454]:
sol = []
if model.SolCount > 0:
    for c in range(m):
        sol.append(find_path(paths,c))

In [455]:
json_dict = {}
json_dict['MIP'] = {}
json_dict['MIP']['time'] = int(floor(model.Runtime + preprocessing_time.seconds)) if model.SolCount > 0 else time_limit
json_dict['MIP']['optimal'] = True if (model.Runtime + preprocessing_time.seconds + safe_bound < time_limit) else False
json_dict['MIP']['obj'] = int(model.ObjVal) if model.SolCount > 0 else None
json_dict['MIP']['sol'] = sol

with open(f'res/MIP/{str(int(innum))}.json', 'w') as outfile:
    json.dump(json_dict, outfile)